In [1]:
# Pakiety
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import random
import warnings 
warnings.filterwarnings('ignore')
# Ustawienie ziarna w celu gwarancji reprodukowalności wyników
np.random.seed(123) 

# Link do wybranego zbioru: https://www.openml.org/d/1448

In [2]:
# Wczytanie danych
apartments = pd.read_csv('apartments.csv')
knugget_chase = pd.read_csv('KnuggetChase3.csv')
knugget_chase.rename(columns={'def':'target'}, inplace=True)

## 1. Zbiór Apartments

In [3]:
print('\033[1m' + 'Apartments dataset:' + '\033[0m')
apartments.info()

Apartments dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
m2.price             1000 non-null int64
construction.year    1000 non-null int64
surface              1000 non-null int64
floor                1000 non-null int64
no.rooms             1000 non-null int64
district             1000 non-null object
dtypes: int64(5), object(1)
memory usage: 47.0+ KB


Dane nie zawierają braków i nie wymagają czyszczenia.

In [4]:
# Sprawdzenie liczności i balansu klas predykcyjnych w poszczególnych zbiorach
print('\033[1m' + 'Apartments dataset:' + '\033[0m')
apartments.district.value_counts()

Apartments dataset:


Mokotow        107
Wola           106
Ursus          105
Ursynow        103
Srodmiescie    100
Bemowo          98
Zoliborz        97
Ochota          96
Bielany         96
Praga           92
Name: district, dtype: int64

Zatem klasyfikacja ma charakter wieloklasowy, zmienna celu jest zbalansowana.

Dla klasyfikacji wieloklasowej SVM wdraża strategię OVO (one-vs-one), polegającą  na  konstruowaniu  klasyfikatorów  dla  wszystkich mozliwych  par  klas.

### Podział na zbiory treningowy i testowy  

* 80% obserwacji należy do zbioru treningowego, pozostałe do testowego.  
* Odseparowujemy zmienną celu od zmiennych objaśniających.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(apartments.drop('district', axis=1), apartments['district'])

### 1.1. Dopasowanie SVM

In [6]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [7]:
y_pred = clf.predict(X_test)

### Rezultaty

In [8]:
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

      Bemowo      0.000     0.000     0.000        21
     Bielany      0.000     0.000     0.000        29
     Mokotow      0.187     0.467     0.267        30
      Ochota      0.000     0.000     0.000        25
       Praga      0.000     0.000     0.000        24
 Srodmiescie      0.528     0.950     0.679        20
       Ursus      0.133     0.320     0.188        25
     Ursynow      0.000     0.000     0.000        23
        Wola      0.141     0.310     0.194        29
    Zoliborz      0.000     0.000     0.000        24

    accuracy                          0.200       250
   macro avg      0.099     0.205     0.133       250
weighted avg      0.094     0.200     0.128       250



In [9]:
np.unique(y_pred)

array(['Mokotow', 'Srodmiescie', 'Ursus', 'Ursynow', 'Wola'], dtype=object)

Przeciętne wyniki, z 10 wszystkich możliwych klas zmiennej odpowiedzi, w predykcji uwzględnionych zostało tylko 5. 

### 1.2. Skalowanie

In [10]:
# Kolumny z danymi numerycznymi
num = apartments.select_dtypes(include=['float64', 'int']).columns
# Skalowanie danych numerycznych
scaler = StandardScaler()
apartments[num] = scaler.fit_transform(apartments[num])

In [11]:
# Podział na zbiory testowy i walidacyjny
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(apartments.drop('district', axis=1), apartments['district'])

In [12]:
clf_scaled = svm.SVC()
clf_scaled.fit(X_scaled_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
y_scaled_pred = clf_scaled.predict(X_scaled_test)

### Rezultaty

In [14]:
print(metrics.classification_report(y_test, y_scaled_pred, digits=3))

              precision    recall  f1-score   support

      Bemowo      0.227     0.185     0.204        27
     Bielany      0.174     0.143     0.157        28
     Mokotow      0.300     0.375     0.333        24
      Ochota      0.400     0.214     0.279        28
       Praga      0.000     0.000     0.000        23
 Srodmiescie      1.000     1.000     1.000        24
       Ursus      0.229     0.333     0.271        24
     Ursynow      0.158     0.231     0.187        26
        Wola      0.121     0.154     0.136        26
    Zoliborz      0.321     0.450     0.375        20

    accuracy                          0.300       250
   macro avg      0.293     0.309     0.294       250
weighted avg      0.290     0.300     0.289       250



In [15]:
np.unique(y_scaled_pred)

array(['Bemowo', 'Bielany', 'Mokotow', 'Ochota', 'Praga', 'Srodmiescie',
       'Ursus', 'Ursynow', 'Wola', 'Zoliborz'], dtype=object)

Skalowanie zmiennych wywarło duży wpływ (zdecydowanie pozytywny) na jakość predykcji - poprawiło rezultaty względem rozważanych miar, wyeliminowało problem nieuwzględnienia w predykcji większości klas zmiennej celu.

### 1.3. Optymalizacja hiperparametrów dla jądra gaussowskiego

In [16]:
from scipy.stats import uniform as sp_uniform

# Utworzenie siatki hiperparametrów
C_range = sp_uniform(scale=10)
gamma_range = sp_uniform(scale=1)
degree_range = sp_uniform(scale = 20)

params1 = {'kernel': ['rbf'],
              'C':C_range, 
              'gamma': gamma_range,
              'degree': degree_range
 }

In [17]:
svm_clsf = svm.SVC()
rnd_clsf = RandomizedSearchCV(estimator=svm_clsf,
                              param_distributions=params1,
                              n_iter=1, 
                              cv=3,
                              n_jobs=-1,
                              verbose=2)
rnd_clsf.fit(X_scaled_train, y_train) 
rnd_clsf.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.0s finished


{'C': 1.3995143199050641,
 'degree': 16.131066700773122,
 'gamma': 0.9581798185048573,
 'kernel': 'rbf'}

In [18]:
best_clsf = rnd_clsf.best_estimator_
best_clsf.fit(X_scaled_train, y_train)
best_clsf_pred = best_clsf.predict(X_scaled_test)

### Rezultaty

In [19]:
print(metrics.classification_report(y_test, best_clsf_pred, digits=3))

              precision    recall  f1-score   support

      Bemowo      0.167     0.185     0.175        27
     Bielany      0.200     0.214     0.207        28
     Mokotow      0.200     0.208     0.204        24
      Ochota      0.409     0.321     0.360        28
       Praga      0.222     0.087     0.125        23
 Srodmiescie      1.000     0.958     0.979        24
       Ursus      0.154     0.167     0.160        24
     Ursynow      0.154     0.154     0.154        26
        Wola      0.194     0.269     0.226        26
    Zoliborz      0.304     0.350     0.326        20

    accuracy                          0.288       250
   macro avg      0.300     0.291     0.292       250
weighted avg      0.297     0.288     0.288       250



In [20]:
np.unique(best_clsf_pred)

array(['Bemowo', 'Bielany', 'Mokotow', 'Ochota', 'Praga', 'Srodmiescie',
       'Ursus', 'Ursynow', 'Wola', 'Zoliborz'], dtype=object)

Uwaga!  
Należy wziąć pod uwagę fakt, że powyższy estymator wykorzystuje dane przeskalowane (w przeciwnym razie porównanie między estymatorami byłoby niezasadne).    

Wbrew intuicji, okazuje się, że w porównaniu z poprzednim estymatorem wyniki precision i recall są w ogólności gorsze, chociaż nie jest to jednoznaczna tendencja. 

### 1.4. Optymalizacja hiperparametrów z uwzględnieniem innych jąder

In [21]:
params2 = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'C':C_range, 
              'gamma': gamma_range,
              'degree': degree_range
 }

svm_clsf = svm.SVC()
rnd_clsf = RandomizedSearchCV(estimator=svm_clsf,
                              param_distributions=params2,
                              n_iter=1, 
                              cv=3,
                              n_jobs=-1,
                              verbose=2)
rnd_clsf.fit(X_scaled_train, y_train) 
rnd_clsf.best_params_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.6s finished


{'C': 0.8855097432789794,
 'degree': 11.799699607422635,
 'gamma': 0.1292164999154305,
 'kernel': 'poly'}

In [22]:
best_clsf = rnd_clsf.best_estimator_
best_clsf.fit(X_scaled_train, y_train)
best_clsf_pred = best_clsf.predict(X_scaled_test)

### Rezultaty

In [23]:
print(metrics.classification_report(y_test, best_clsf_pred, digits=3))

              precision    recall  f1-score   support

      Bemowo      0.000     0.000     0.000        27
     Bielany      0.500     0.036     0.067        28
     Mokotow      0.250     0.250     0.250        24
      Ochota      0.500     0.071     0.125        28
       Praga      0.000     0.000     0.000        23
 Srodmiescie      1.000     0.375     0.545        24
       Ursus      0.110     0.917     0.196        24
     Ursynow      0.000     0.000     0.000        26
        Wola      0.167     0.038     0.062        26
    Zoliborz      0.000     0.000     0.000        20

    accuracy                          0.164       250
   macro avg      0.253     0.169     0.125       250
weighted avg      0.260     0.164     0.123       250



In [24]:
np.unique(best_clsf_pred)

array(['Bemowo', 'Bielany', 'Mokotow', 'Ochota', 'Praga', 'Srodmiescie',
       'Ursus', 'Ursynow', 'Wola', 'Zoliborz'], dtype=object)

Rezultaty w ogólności gorsze niż w dwóch poprzednich przypadkach.   

**Zatem najlepszy wynik, dla zbioru Apartments, uzyskaliśmy dla niestrojonego estymatora na danych skalowanych.**

## 2. Zbiór Knugget Chase

In [25]:
print('\n \033[1m' + 'Knugget Chase dataset:' + '\033[0m')
knugget_chase.info()


 Knugget Chase dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 40 columns):
a         194 non-null int64
b         194 non-null int64
c         194 non-null int64
d         194 non-null int64
e         194 non-null int64
f         194 non-null int64
g         194 non-null int64
h         194 non-null float64
i         194 non-null int64
j         194 non-null float64
k         194 non-null int64
l         194 non-null float64
m         194 non-null int64
n         194 non-null int64
o         194 non-null float64
p         194 non-null int64
r         194 non-null int64
s         194 non-null int64
t         194 non-null float64
u         194 non-null float64
v         194 non-null float64
z         194 non-null float64
aa        194 non-null float64
ab        194 non-null int64
ac        194 non-null float64
ad        194 non-null float64
ae        194 non-null float64
af        194 non-null float64
ag        194 non-null int64
ah 

Jak wynika z opisu jedyną zmienną nominalną w zbiorze jest zmienna celu, wszystkie pozostałe mają charakter numeryczny.  
Dane nie zawierają braków i nie wymagają czyszczenia.

In [26]:
# Sprawdzenie liczności i balansu klas predykcyjnych w poszczególnych zbiorach
print('\033[1m' + 'Knugget Chase dataset:' + '\033[0m')
knugget_chase.target.value_counts()

Knugget Chase dataset:


N    158
Y     36
Name: target, dtype: int64

Zatem, klasyfikacja ma charakter binarny. Ponadto zmienna celu jest niezbalansowana. Należy to wziąć pod uwagę przy doborze miar względem, których oceniana będzie predykcja.

In [27]:
# Podział na zbiór treningowy i walidacyjny, analogiczniy jak w poprzednim przypadku.
X_train, X_test, y_train, y_test = train_test_split(knugget_chase.drop('target', axis=1), knugget_chase['target'])

### 2.1. Dopasowanie SVM

In [28]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
y_pred = clf.predict(X_test)

### Rezultaty

In [30]:
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           N      0.673     1.000     0.805        33
           Y      0.000     0.000     0.000        16

    accuracy                          0.673        49
   macro avg      0.337     0.500     0.402        49
weighted avg      0.454     0.673     0.542        49



Dane są niezbalandowane, dlatego do oceny jakości estymatora bardziej wartościowe od accuracy są miary precision i recall. Nie mniej jednak dla zachowania konsekwencji w obrębie raportu za każdym razem uwzględniam ją w sekcji z wynikami.  
Dla powyższego estymatora zauważalna jest duża rozbieżność wyników. Precision znacznie odbiega od recall, które w efekcie, sztucznie zawyża F1.

### 2.2. Skalowanie

In [31]:
# Kolumny z danymi numerycznymi
num = knugget_chase.select_dtypes(include=['float64', 'int']).columns
# Skalowanie danych numerycznych
scaler = StandardScaler()
knugget_chase[num] = scaler.fit_transform(knugget_chase[num])

In [32]:
# Podział na zbiory testowy i walidacyjny
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(knugget_chase.drop('target', axis=1), knugget_chase['target'])

In [33]:
clf_scaled = svm.SVC()
clf_scaled.fit(X_scaled_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [34]:
y_scaled_pred = clf_scaled.predict(X_scaled_test)

### Rezultaty

In [35]:
print(metrics.classification_report(y_test, y_scaled_pred, digits=3))

              precision    recall  f1-score   support

           N      0.816     1.000     0.899        40
           Y      0.000     0.000     0.000         9

    accuracy                          0.816        49
   macro avg      0.408     0.500     0.449        49
weighted avg      0.666     0.816     0.734        49



W wyniku skalowania zmiennych, wyniki ponownie uległy znacznej poprawie - doszło do znacznego podwyższenia precision przy zachowaniu rezultatu recall. 

### 2.3. Optymalizacja hiperparametrów dla jądra gaussowskiego

In [120]:
svm_clsf = svm.SVC()
rnd_clsf = RandomizedSearchCV(estimator=svm_clsf,
                              param_distributions=params1,
                              n_iter=1, 
                              cv=3,
                              n_jobs=-1,
                              verbose=2)
rnd_clsf.fit(X_scaled_train, y_train)
rnd_clsf.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


{'C': 2.205710171010927,
 'degree': 17.248117945818485,
 'gamma': 0.91146646662787,
 'kernel': 'rbf'}

In [121]:
best_clsf = rnd_clsf.best_estimator_
best_clsf.fit(X_scaled_train, y_train)
best_clsf_pred = best_clsf.predict(X_scaled_test)

### Rezultaty

In [122]:
print(metrics.classification_report(y_test, best_clsf_pred, digits=3))

              precision    recall  f1-score   support

           N      0.816     1.000     0.899        40
           Y      0.000     0.000     0.000         9

    accuracy                          0.816        49
   macro avg      0.408     0.500     0.449        49
weighted avg      0.666     0.816     0.734        49



Analogicznie jak w dla poprzedniego zbioru, estymator strojony korzysta z danych skalowanych.  
W tym przypadku jednak, inaczej niż poprzednio strojenie nie zmieniło rezultatów.  
Mając w świadomości losowość strojenia, wielokrotnie je powtórzyłam i powyższy wynik, jednakowy jak bez strojenia, jest najlepszym spośród uzyskanych.

### 2.4. Optymalizacja hiperparametrów z uwzględnieniem innych jąder

In [132]:
svm_clsf = svm.SVC()
rnd_clsf = RandomizedSearchCV(estimator=svm_clsf,
                              param_distributions=params2,
                              n_iter=1, 
                              cv=3,
                              n_jobs=-1,
                              verbose=2)
rnd_clsf.fit(X_scaled_train, y_train) 
rnd_clsf.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


{'C': 3.380000380848366,
 'degree': 7.369063458939699,
 'gamma': 0.7695895209567643,
 'kernel': 'poly'}

In [133]:
best_clsf = rnd_clsf.best_estimator_
best_clsf.fit(X_scaled_train, y_train)
best_clsf_pred = best_clsf.predict(X_scaled_test)

### Rezultaty

In [134]:
print(metrics.classification_report(y_test, best_clsf_pred, digits=3))

              precision    recall  f1-score   support

           N      0.860     0.925     0.892        40
           Y      0.500     0.333     0.400         9

    accuracy                          0.816        49
   macro avg      0.680     0.629     0.646        49
weighted avg      0.794     0.816     0.801        49



Wyniki w ogólności gorsze niż uzyskane dla dwóch poprzednich estymatorów.  
  
**Najlepsze rezultaty, dla zbioru Knugget Chase, uzyskano dla dla niestrojonego estymatora oraz strojonego estymatora o jądrze gaussowskim wykorzystujących dane skalowane**.  